In [1]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'iavVcVqTHtrhyMOqsMSuTdYZCqaa94Jpyu3zzTJoubTi',
    'service_id': 'iam-ServiceId-413d609d-bb8b-4322-903b-1c62138667c6',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_281ff1d7223247ceb85a02769ab11f1a_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initialized for you.
# The following variable contains the path to your file on your IBM Cloud Object Storage.
path_1 = cos.url('user.json.bz2', 'spring2018veda7abe03040343431897343076417c41c8')


# Read Review Data
Read the review data from round 11 of the Yelp Dataset challenge. The 5.2 million reviews in this data file include every review from the following 11 cities, except those reviews that Yelp has filtered as not being recommended. 

* Las Vegas, NV
* Phoenix, AZ
* Madison, WI
* Urbana-Champagne, IL
* Cleveland, OH
* Pittsburgh, PA
* Charlotte, NC
* Montreal (Canada)
* Quebec (Canada)
* Edinburg (U.K.)
* Karlsruhe, (Germany)


In [38]:

# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initialized for you.
# The following variable contains the path to your file on your IBM Cloud Object Storage.
path_reviews = cos.url('review.json.bz2', 'spring2018veda7abe03040343431897343076417c41c8')
df_reviews = spark.read.json(path_reviews)
print "number of reviews:", df_reviews.count()
df_reviews.printSchema()
df_reviews.show(5)
df_reviews.select("business_id", "user_id", "review_id").createOrReplaceTempView("reviews")



number of reviews: 5261669
root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|0W4lkclzZThpx3V65...|   0|2016-05-28|    0|v0i_UHJMo_hPBq9bx...|    5|Love the staff, l...|     0|bv2nCi5Qv5vroFiqK...|
|AEx2SYEUJmTxVVB18...|   0|2016-05-28|    0|vkVSCC7xljjrAI4UG...|    5|Super simple plac...|     0|bv2nCi5Qv5vroFiqK...|
|VR6GpWIda3SfvPC-l...|   0|

# Read User Data
This data includes every user that wrote a review in any of the 11 cities in the review data. 
Please note, the review data does not include every review from each of these users. 



In [3]:
path_user = cos.url('user.json.bz2', 'spring2018veda7abe03040343431897343076417c41c8')
df_users = spark.read.json(path_user)
print "number of users:", df_users.count()
df_users.printSchema()
df_users.show(5, truncate=False)
df_users.select("review_count","user_id").createOrReplaceTempView("users")

number of users: 1326101
root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- fans: long (nullable = true)
 |-- friends: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true

In [4]:
from pyspark.sql import functions
df_filtered_reviews=df_reviews.select(df_reviews.review_id,df_reviews.user_id,df_reviews.business_id,\
                                      df_reviews.stars,functions.length(df_reviews.text).alias('review_length'))
df_filtered_reviews.show(5)



+--------------------+--------------------+--------------------+-----+-------------+
|           review_id|             user_id|         business_id|stars|review_length|
+--------------------+--------------------+--------------------+-----+-------------+
|v0i_UHJMo_hPBq9bx...|bv2nCi5Qv5vroFiqK...|0W4lkclzZThpx3V65...|    5|          289|
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|          213|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|    5|          502|
|MV3CcKScW05u5LVfF...|bv2nCi5Qv5vroFiqK...|CKC0-MOWMqoeWf6s-...|    5|          373|
|IXvOzsEMYtiJI0CAR...|bv2nCi5Qv5vroFiqK...|ACFtxLv8pGrrxMm6E...|    4|          523|
+--------------------+--------------------+--------------------+-----+-------------+
only showing top 5 rows



In [5]:
df_filtered_users=df_users.select(df_users.user_id,df_users.name,df_users.review_count,df_users.average_stars,\
                                 df_users.yelping_since,(functions.size(df_users.elite)>0).alias('is_elite'))
df_filtered_users.show(5)

+--------------------+------+------------+-------------+-------------+--------+
|             user_id|  name|review_count|average_stars|yelping_since|is_elite|
+--------------------+------+------------+-------------+-------------+--------+
|oMy_rEb0UBEmMlu-z...|Johnny|           8|         4.67|   2014-11-03|   false|
|JJ-aSuM4pCFPdkfoZ...| Chris|          10|          3.7|   2013-09-24|   false|
|uUzsFQn_6cXDh6rPN...| Tiffy|           1|          2.0|   2017-03-02|   false|
|mBneaEEH5EMyxaVyq...|  Mark|           6|         4.67|   2015-03-13|   false|
|W5mJGs-dcDWRGEhAz...|Evelyn|           3|         4.67|   2016-09-08|   false|
+--------------------+------+------------+-------------+-------------+--------+
only showing top 5 rows



In [6]:
df_agg_rev=df_filtered_reviews.groupBy('user_id').agg({'stars':'avg','review_length':'avg','review_id':'count'})
df_agg_rev.show(5)

df_agg_reviews=df_agg_rev.select('user_id','avg(stars)','avg(review_length)','count(review_id)')\
.toDF('user_id','dataset_avg_stars','avg_review_length','dataset_review_count')
df_agg_reviews.show(5)



+--------------------+----------------+-----------------+------------------+
|             user_id|count(review_id)|       avg(stars)|avg(review_length)|
+--------------------+----------------+-----------------+------------------+
|oKWVVqPWVzq5s6nS4...|              42|4.190476190476191|370.14285714285717|
|OcB8_br3QOUuX_CCg...|               3|              4.0|233.66666666666666|
|5a7nRCjFm_QUjF8xy...|               1|              1.0|             408.0|
|DsPTtxXzLccDutfoJ...|               9|              5.0|             447.0|
|Y1N3yo62V5eIj_Zcn...|               1|              5.0|              95.0|
+--------------------+----------------+-----------------+------------------+
only showing top 5 rows

+--------------------+-----------------+------------------+--------------------+
|             user_id|dataset_avg_stars| avg_review_length|dataset_review_count|
+--------------------+-----------------+------------------+--------------------+
|oKWVVqPWVzq5s6nS4...|4.190476190476191

In [7]:
df_users_reviews=df_filtered_users.join(df_agg_reviews,'user_id','left')\
.select(df_filtered_users.user_id,'name','review_count','average_stars',\
       'yelping_since','is_elite','dataset_avg_stars','avg_review_length','dataset_review_count')
df_users_reviews.show(5)

+--------------------+-------------+------------+-------------+-------------+--------+-----------------+-----------------+--------------------+
|             user_id|         name|review_count|average_stars|yelping_since|is_elite|dataset_avg_stars|avg_review_length|dataset_review_count|
+--------------------+-------------+------------+-------------+-------------+--------+-----------------+-----------------+--------------------+
|--CJT4d-S8UhwqHe0...|        Scott|           3|          2.0|   2015-08-18|   false|              1.0|           1956.0|                   1|
|-0Ji0nOyFe-4yo8BK...|         cubs|          57|         3.28|   2009-05-15|   false|              4.0|             64.0|                   1|
|-0XPr1ilUAfp-yIXZ...|Fairmount Jil|           3|         1.25|   2007-12-26|   false|              2.0|           1849.0|                   1|
|-1KKYzibGPyUX-Mwk...|       Nickie|         163|          4.0|   2009-08-07|    true|              4.0|            510.0|              

In [8]:
df_dataset_percentage=df_users_reviews.select('user_id','name','review_count','average_stars','yelping_since',\
                                             'is_elite','dataset_avg_stars','avg_review_length','dataset_review_count',\
                                             (df_users_reviews.dataset_review_count/df_users_reviews.review_count)\
                                             .alias('dataset_percentage'))
df_dataset_percentage.show()

+--------------------+-------------+------------+-------------+-------------+--------+------------------+------------------+--------------------+--------------------+
|             user_id|         name|review_count|average_stars|yelping_since|is_elite| dataset_avg_stars| avg_review_length|dataset_review_count|  dataset_percentage|
+--------------------+-------------+------------+-------------+-------------+--------+------------------+------------------+--------------------+--------------------+
|--CJT4d-S8UhwqHe0...|        Scott|           3|          2.0|   2015-08-18|   false|               1.0|            1956.0|                   1|  0.3333333333333333|
|-0Ji0nOyFe-4yo8BK...|         cubs|          57|         3.28|   2009-05-15|   false|               4.0|              64.0|                   1|0.017543859649122806|
|-0XPr1ilUAfp-yIXZ...|Fairmount Jil|           3|         1.25|   2007-12-26|   false|               2.0|            1849.0|                   1|  0.3333333333333333

# Read Business data
This data includes all data related to the businesses within the 2017 Yelp dataset. Also created a temporary view from the business data called business. 

In [9]:

# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initialized for you.
# The following variable contains the path to your file on your IBM Cloud Object Storage.
path_business = cos.url('business.json.bz2', 'spring2018veda7abe03040343431897343076417c41c8')
df_business= spark.read.json(path_business)
df_business.show()
df_business.select("business_id","name","state","city","is_open","review_count","categories").createOrReplaceTempView("business")


+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|     longitude|                name|      neighborhood|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+------------------+-----------+------------+-----+-----+
|4855 E Warner Rd,...|[true,null,null,n...|FYWN1wneV18bWNgQj...|[Dentists, Genera...|     Ahwatukee|[7:30-17:00,7:30-...|      1|   33.3306902|  -111.9785992|    Dental by Design|                  |      85044|          22|  4.0|   AZ|
|  3101 Washington Rd|[null,null,null,n...|He-G7vWjzVUys

# How many businesses are there in each of the nine North American Cities?

- Created a subquery to combine South Carolina and North Carolina since they are a part of the same Metropolitan area. 
    * The subquery creates a temporary table with the outputs of each state
    * Summing the subquery's output gives us the total for NC and SC combined. 
- Finally we counted how many distinct overall businesses there are in the nine North American Cities.

In [10]:
spark.sql("""
SELECT state,
SUM(number_of_businesses) as number_of_businesses
FROM
(SELECT (CASE when state = "NC" or state = "SC" then "NC" else state end) as state,
        COUNT(DISTINCT name) as number_of_businesses
        FROM business
        WHERE 
            state = "AZ" OR
            state = "IL" OR
            state = "NV" OR
            state = "OH" OR
            state = "ON" OR
            state = "PA" OR
            state = "QC" OR
            state = "SC" OR
            state = "NC" 
        GROUP BY state)
GROUP BY state
""").show()


+-----+--------------------+
|state|number_of_businesses|
+-----+--------------------+
|   AZ|               40197|
|   QC|                7089|
|   NV|               26309|
|   NC|               10734|
|   IL|                1654|
|   OH|                9732|
|   PA|                8189|
|   ON|               23741|
+-----+--------------------+



#  How many continuing locals are there in the data set?       
* Created a temporary view for data frame for continuing users from the dataset percentage dataframe
* The dataset percentage dataframe holds data from the users data so we got a count of each name
* We defined locals as users who had at least 10 reviews and had 50% of their reviews and then filtered the data to only include those

In [11]:
df_dataset_percentage.createOrReplaceTempView("continuing_users")
spark.sql("""
SELECT COUNT(user_id) as continuing_users
FROM continuing_users
WHERE 
     review_count>= 10 AND dataset_percentage>0.5
""").show()

+----------------+
|continuing_users|
+----------------+
|           88301|
+----------------+



# How many reviews are there in each of the nine cities?
- Created a subquery to combine South Carolina and North Carolina since they are a part of the same Metropolitan area. 
    * The subquery creates a temporary table with the outputs of each state
    * Summing the subquery's output gives us the total for NC and SC combined. 
- Finally we counted how many reviews there are in the nine North American cities.

In [73]:
df_filtered_states = spark.sql("""
SELECT *,
    CASE
        WHEN state = "SC" THEN "NC"
        ELSE
          state
    END AS s
FROM business
WHERE     
    state = "AZ" OR
    state = "IL" OR
    state = "WI" OR
    state = "NV" OR
    state = "OH" OR
    state = "ON" OR
    state = "PA" OR
    state = "QC" OR
    state = "SC" OR
    state = "NC" 
""")

df_filtered_states.show()

+--------------------+--------------------+-----+--------------+-------+------------+--------------------+---+
|         business_id|                name|state|          city|is_open|review_count|          categories|  s|
+--------------------+--------------------+-----+--------------+-------+------------+--------------------+---+
|FYWN1wneV18bWNgQj...|    Dental by Design|   AZ|     Ahwatukee|      1|          22|[Dentists, Genera...| AZ|
|He-G7vWjzVUysIKrf...| Stephen Szabo Salon|   PA|      McMurray|      1|          11|[Hair Stylists, H...| PA|
|KQPW8lFf1y5BT2Mxi...|Western Motor Veh...|   AZ|       Phoenix|      1|          18|[Departments of M...| AZ|
|8DShNS-LuFqpEWIp0...|    Sports Authority|   AZ|         Tempe|      0|           9|[Sporting Goods, ...| AZ|
|PfOCPjBrlQAnz__NX...|Brick House Taver...|   OH|Cuyahoga Falls|      1|         116|[American (New), ...| OH|
|kCoE3jvEtg6UVz5SO...|          BDJ Realty|   NV|     Las Vegas|      1|           5|[Real Estate Serv...| NV|
|

In [74]:
df_filtered_states.createOrReplaceTempView("filtered_states")

df_reviews_by_state=spark.sql("""
SELECT state, SUM(review_count) AS number_of_reviews
FROM filtered_states
GROUP BY state
""")

df_reviews_by_state.show()

df_reviews_by_state.select("state","number_of_reviews").createOrReplaceTempView("state_review_count")


+-----+-----------------+
|state|number_of_reviews|
+-----+-----------------+
|   AZ|          1627693|
|   SC|            10857|
|   QC|           146379|
|   NV|          1824387|
|   WI|           109737|
|   NC|           307426|
|   IL|            36378|
|   OH|           243677|
|   PA|           229804|
|   ON|           634263|
+-----+-----------------+



# Do some cities have a higher share of reviews written by locals who are continuing Yelpers?

We combined user, review, and business data in order to attach state data to users. This allowed us to make continuing user data accessible for each metro area

# Finding all the reviews written by local users

We used the dataset_percentage dataframe to make a new dataframe listing all the local users and their review counts. 

In [14]:
df_dataset_percentage.createOrReplaceTempView("dataset_percentage")
df_local_users = spark.sql("""
SELECT user_id, dataset_review_count, dataset_percentage
FROM dataset_percentage 
WHERE dataset_review_count >= 10 AND dataset_percentage > 0.5
""")
df_local_users.show()

df_local_users.createOrReplaceTempView("local_users")

+--------------------+--------------------+------------------+
|             user_id|dataset_review_count|dataset_percentage|
+--------------------+--------------------+------------------+
|-55DgUo52I3zW9Rxk...|                  22|0.5945945945945946|
|-NnADpikTs6IWM9tb...|                  21|0.7241379310344828|
|-dErbI4sHSkRz6oxj...|                  28|0.6086956521739131|
|-vSUDEQB9j2DaP8_4...|                  25|0.7575757575757576|
|0-UxxxWLz1muOzPx2...|                  29|0.6744186046511628|
|0BMTo253SdSZpmZbA...|                  12|0.6666666666666666|
|0LUt7xSvYgLpL0zBe...|                  16|0.9411764705882353|
|0M4x1qpXH1-Ssuepp...|                  20|0.7142857142857143|
|1CHxxE-_z3yLsoHq4...|                  16|               1.0|
|1Q-tjPYDBaFZJouj8...|                  20|0.7407407407407407|
|1Qo3Pw-XRPaaNfpGP...|                  54|0.8709677419354839|
|1dI8Gi0M_6RM0ylnA...|                  12|               0.8|
|1oBWrKCDvH221hueH...|                  30|0.8571428571

In [15]:
print df_local_users.count()

67846


# Joining Business data with Reviews and Users
At this point we figured out that we needed a common point in which we could connect all our temp views and dataframe. We knew we had to start we reviews as it was the "middle man" between business and users. So we did just that we connect business_id, review_id, user_id. This would allow us to get closer to finding the percentage of total local users by state.

In [39]:
df_business_reviews=spark.sql("""
SELECT business.business_id, state, user_id, review_id
FROM business, reviews
WHERE business.business_id = reviews.business_id
""")

df_business_reviews.show()

df_business_reviews.createOrReplaceTempView("business_reviews")


+--------------------+-----+--------------------+--------------------+
|         business_id|state|             user_id|           review_id|
+--------------------+-----+--------------------+--------------------+
|0W4lkclzZThpx3V65...|   QC|bv2nCi5Qv5vroFiqK...|v0i_UHJMo_hPBq9bx...|
|AEx2SYEUJmTxVVB18...|   QC|bv2nCi5Qv5vroFiqK...|vkVSCC7xljjrAI4UG...|
|VR6GpWIda3SfvPC-l...|   QC|bv2nCi5Qv5vroFiqK...|n6QzIUObkYshz4dz2...|
|CKC0-MOWMqoeWf6s-...|   QC|bv2nCi5Qv5vroFiqK...|MV3CcKScW05u5LVfF...|
|ACFtxLv8pGrrxMm6E...|   QC|bv2nCi5Qv5vroFiqK...|IXvOzsEMYtiJI0CAR...|
|s2I_Ni76bjJNK9yG6...|   QC|bv2nCi5Qv5vroFiqK...|L_9BTb55X0GDtThi6...|
|8QWPlVQ6D-OExqXoa...|   NV|_4iMDXbXZ1p1ONG29...|HRPm3vEZ_F-33TYVT...|
|9_CGhHMz8698M9-Pk...|   ON|u0LXt3Uea_GidxRW1...|ymAUG8DZfQcFTBSOi...|
|gkCorLgPyQLsptTHa...|   ON|u0LXt3Uea_GidxRW1...|8UIishPUD92hXtScS...|
|5r6-G9C4YLbC7Ziz5...|   ON|u0LXt3Uea_GidxRW1...|w41ZS9shepfO3uEyh...|
|fDF_o2JPU8BR1Gya-...|   ON|u0LXt3Uea_GidxRW1...|WF_QTN3p-thD74hqp...|
|z8oIo

# Joining combined Business+ Reviews dataframe with Local Users dataframe
Once we joined business and review data, we had a user_id to create a join with the local users dataframe from earlier. This created a dataframe with all the reviews written by local users. 

In [41]:
df_business_user_join= spark.sql("""
SELECT business_id, state, local_users.user_id, review_id
FROM business_reviews, local_users
WHERE business_reviews.user_id = local_users.user_id
""")
df_business_user_join.show()

df_business_user_join.createOrReplaceTempView("join_business_user")

+--------------------+-----+--------------------+--------------------+
|         business_id|state|             user_id|           review_id|
+--------------------+-----+--------------------+--------------------+
|WXR0ND0KqbArMZDvF...|   AZ|-55DgUo52I3zW9Rxk...|N_FpnV61L6-6Uf-FV...|
|3qlqzQrwh8hjBltlg...|   AZ|-55DgUo52I3zW9Rxk...|NFgnyyp2Kl1nlWIEk...|
|nn4ZkIZTCd7jJhxs-...|   AZ|-55DgUo52I3zW9Rxk...|zqwxKCkD2PfTRRft5...|
|RezqChEv5DPJC85d2...|   AZ|-55DgUo52I3zW9Rxk...|dqY4HJ4GMwaHsTZUB...|
|_e3rChyednoh47n2c...|   AZ|-55DgUo52I3zW9Rxk...|KW_RA0roCySRdIHCP...|
|Mypk1XBD6PQ77zLXy...|   AZ|-55DgUo52I3zW9Rxk...|9m6mrz0QsbHKHiyM_...|
|F-QY0xyNi9z1p0wjZ...|   AZ|-55DgUo52I3zW9Rxk...|vyUmRDInsAeAvocHV...|
|qrK73viBBJY_hx3bg...|   AZ|-55DgUo52I3zW9Rxk...|_pMYiNeCJtFbCLLVF...|
|9O-L6F0cMfNmE5i07...|   AZ|-55DgUo52I3zW9Rxk...|iOTTR1df4vYAtE9Wn...|
|T-KniGykrZ46ZC9pl...|   AZ|-55DgUo52I3zW9Rxk...|EtVmDwvpG4BLkorpK...|
|DaVTuhzi6EgWStb2e...|   AZ|-55DgUo52I3zW9Rxk...|QKqwHltxRwC4xC6XX...|
|LIxNf

# Finding Local Users by State
We already found the aggregated sum of total local users for all states, but for this assignment we have to find local users and assign them to their respective states, to then be able to find the percentage of local reviews from by each city.

In [45]:
df_continuing_users_by_state=spark.sql("""
SELECT state,
SUM(number_of_local_reviews) as number_of_local_reviews
FROM
    (SELECT (CASE when state = "NC" or state = "SC" then "NC" else state end) as state,
        COUNT(*) as number_of_local_reviews
        FROM join_business_user
WHERE 
    state = "AZ" OR
    state = "IL" OR
    state = "NV" OR
    state = "WI" OR
    state = "OH" OR
    state = "ON" OR
    state = "PA" OR
    state = "QC" OR
    state = "SC" OR
    state = "NC" 
GROUP BY state)
GROUP BY state
""")

df_continuing_users_by_state.show()


+-----+-----------------------+
|state|number_of_local_reviews|
+-----+-----------------------+
|   AZ|                 684274|
|   QC|                  54528|
|   NV|                 564190|
|   WI|                  33140|
|   NC|                 107315|
|   IL|                   6654|
|   OH|                  83512|
|   PA|                  76107|
|   ON|                 357677|
+-----+-----------------------+



In [69]:
df_continuing_users_by_state.createOrReplaceTempView("continuing_users_by_state")

In [76]:
df_compare_local= spark.sql("""
SELECT state_review_count.state, state_review_count.number_of_reviews,continuing_users_by_state.number_of_local_reviews
FROM state_review_count, continuing_users_by_state
WHERE state_review_count.state = continuing_users_by_state.state
""")
df_compare_local.show()


+-----+-----------------+-----------------------+
|state|number_of_reviews|number_of_local_reviews|
+-----+-----------------+-----------------------+
|   AZ|          1627693|                 684274|
|   QC|           146379|                  54528|
|   NV|          1824387|                 564190|
|   WI|           109737|                  33140|
|   NC|           307426|                 107315|
|   IL|            36378|                   6654|
|   OH|           243677|                  83512|
|   PA|           229804|                  76107|
|   ON|           634263|                 357677|
+-----+-----------------+-----------------------+



In [77]:
df_compare_local.createOrReplaceTempView("compare_local")

spark.sql("""
SELECT *, (number_of_local_reviews/number_of_reviews)*100 AS percentage_of_local_reviews
FROM compare_local
""").show()

+-----+-----------------+-----------------------+---------------------------+
|state|number_of_reviews|number_of_local_reviews|percentage_of_local_reviews|
+-----+-----------------+-----------------------+---------------------------+
|   AZ|          1627693|                 684274|         42.039500077717356|
|   QC|           146379|                  54528|          37.25124505564323|
|   NV|          1824387|                 564190|            30.924907927978|
|   WI|           109737|                  33140|         30.199476931208252|
|   NC|           307426|                 107315|          34.90758751699596|
|   IL|            36378|                   6654|         18.291274946396175|
|   OH|           243677|                  83512|          34.27159723732646|
|   PA|           229804|                  76107|           33.1182224852483|
|   ON|           634263|                 357677|          56.39253748050888|
+-----+-----------------+-----------------------+---------------

# Limitations
Although we were able to achieve a main understanding of what the percentage of each metro area's reviews were written by users who are continuing locals, it is evident that there are limitations of the analysis. The main limitation is involved with who is considered a local user and what metro area they are actually from. We are not guaranteed that the user is local in the metro area because we are using an equation, rather than explicit facts. It is possible that the user may have been on vacation, has moved, or only uses yelp when they are in a certain metro area.